In [105]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import yfinance as yf

In [106]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
# My test spreadsheet values (workbook 'MSKQuickstartTest')
SAMPLE_SPREADSHEET_ID = '16T10gT1AIUgbQl6kHE81rEQ9RChdcaLLk1YCpZ1dQIA'
SAMPLE_RANGE_NAME = 'Sheet1!A2:B'

In [107]:
"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()

In [108]:
values = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()['values']

In [109]:
stocks = [e[0] for e in values]
numshares = [int(e[1]) for e in values]

In [110]:
sts = yf.Tickers(stocks)

In [116]:
sdata = sts.download()

[*********************100%***********************]  12 of 12 completed


In [113]:
sprices = [sdata['Close'][stock].dropna()[-1] for stock in stocks]
values = [[sprice, num * sprice] for sprice, num in zip(sprices, numshares)]


In [114]:
body = {'values': values}

In [115]:
sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range='C2', body=body, valueInputOption='RAW').execute()

{'spreadsheetId': '16T10gT1AIUgbQl6kHE81rEQ9RChdcaLLk1YCpZ1dQIA',
 'updatedRange': 'Sheet1!C2:D13',
 'updatedRows': 12,
 'updatedColumns': 2,
 'updatedCells': 24}